In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import MinMaxScaler 
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import json
import pickle

# Training Pipeline

#### The training process follows an object-oriented approach. All methods and variables included the training process are stored in the class Trainer.

#### The (global) attributes of the class Trainer:
<ol>
    <li>self.census_starter_df: Pandas dataframe containing the the data from the csv file census_starter.csv</li>
    <li>self.train_df: Pandas dataframe containing the the data from the csv file train.csv</li>  
    <li>self.census_columns: Python list containing columns of self.census_starter_df</li>   
    <li>selfcensus_columns_no_date: Python list containing unique column names (no years) from the census dataset self.census_starter_df</li>    
    <li>self.params: Python dictionary containing the hyperparameter space that i used in the hyperparameter optimization during training</li>    
    <li>self.model_container: Empty dictionary that is used to store the trained machine learning model for each county</li>
</ol>

#### The methods of the class Trainer:
<ol>
    <li>__init__(self): Constructor method that is excecuted whenever an object of the class Trainer is created. It initilizes all global attributes of the class Trainer
    </li>
    <li>
        preprocess(self): This method is responsible for the preprocessing steps that are required before the training of a model can take place. It consists of the following methods:
        <ol>
            <li>
                add_census_columns(self): The column names of self.census_starter_df are extracted (pct_bb, pct_college, pct_foreign_born, pct_it_workers, median_hh_inc) and are stored in the list self.census_columns_no_date. The extracted and stored census columns are than added to the training dataset self.census_starter_df with null values
            </li>
            <li>
                add_census_values(self, r): For each row (r) of the dataframe self.train_df, the values of the dataframe self.census_starter_df are retrieved. The matching process between the two dataframes relies on the county code cfips (eg. 1001) and the year that occurs in the column first_day_of_month. Since we are working with census data which are indicators of what will happen in the future, 1 is subtracted from the year in the column first_day_of_month (eg. 2018 - 1 = 2017). In this expample, the values in the columns pct_bb_2017, pct_college_2017, pct_foreign_born_2017, pct_it_workers_2017, and median_hh_in_2017 are retrieved for cfips 1001 and are stored in the columns pct_bb, pct_college, pct_foreign_born, pct_it_workers, and median_hh_inc respectively of the dataframe self.train_df
            </li>
            <li>
                bfill_nan(self): backward fill the missing values in the dataset self.train_df
            </li>
            <li>
                add_month_col(self): Creation of the column Month in the dataset self.train_df retrieved from the column first_day_of_month. This was done in order to capture monthly seasonality within one year
            </li>
            <li>
                remove_non_feature_cols(self): Remove the columns that are not used as input parameters for the training of the model (county, first_day_of_month, state)
            </li>
            <li>
                create_dummies(self): Creation of dummy variables for the column Month. When the Month January is observed, January has the value 1 while the rest of the Months have the value 0. This step is carried out becuase machine learning models work better with numerical values.
            </li>
            <li>
                reindex_dataset(self): Changing the order of the columns in a way that represents the months in ascending order
            </li>
        </ol>
    </li>
    <li>
        train_model(self): For each county code cfips, a seperate xgboost regressor model with its own hyperparameters is trained. The reasons for this approach are the following:
        <ul>
            <li>
                Each county may display unique patterns in the data set. When each county is analyzed individually, the respective patterns are taken into account during the normalization of numerical data which will affect the fitting of the xgboost regressor model
            </li>
            <li>
                Since the dataset has multiple observations for each county code (cfips) in combination with evenly spaced timestamps, the problem at hand can be considered panel data problem. What one can do is to treat the data of each county code as a unique time series. As monthly seasonality is caputured by the columns Month and yearly sasonality by the columns active, pct_bb, pct_college, pct_foreign_born, pct_it_workers, and median_hh_inc, a regression model can be a model of choice for each county code
            </li>
        </ul>
        Therefore, for each county code (cfips):
        <ol>
            <li>
                The columns microbusiness_density, active, pct_bb, pct_college, pct_foreign_born, pct_it_workers, and median_hh_inc are normalized to the values between 0 and 1 with a min-max-scaler (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). This ensures better training of the chosen model and enables the comparison of numerical columns of units of measurement
            </li>
            <li>
                Splitting the data into feature variables (active, pct_bb, pct_college, pct_foreign_born, pct_it_workers, and median_hh_inc) and target (microbusiness_density) variable
            </li>
            <li>
                The best hyperparameter combinations from the dictionary self.params are retrieved via a grid search algorithms
            </li>
            <li>
                The xgboost regressor model is fitted with the feature and target variables and the best hyperparameter combinations from the previous step. The created model is then stored in the dictionary self.model_container where each key represents the county code
            </li>
        </ol>
    </li>
</ol>

In [2]:
class Trainer():
    
    def __init__(self):
        
        self.census_starter_df = pd.read_csv(r"C:\Users\edink\Downloads\godaddy-microbusiness-density-forecasting\census_starter.csv")
        self.train_df = pd.read_csv(r"C:\Users\edink\Downloads\godaddy-microbusiness-density-forecasting\train.csv")
        self.census_columns = self.census_starter_df.columns
        self.census_columns_no_date = []
        self.params = {
            "max_depth": [3, 6, 9],
            "learning_rate": [0.1, 0.3, 0.5],
            "n_estimators": [75, 100, 125],
            }
        self.model_container = {}
        
        
    def preprocess(self):
        self.add_census_columns()
        self.train_df = self.train_df.apply(self.add_census_values, axis = 1)
        self.bfill_nan()
        self.add_month_col()
        self.remove_non_feature_cols()
        self.create_dummies()
        self.reindex_dataset()
        
    
    def add_census_columns(self):
        for col in self.census_columns:
            if col[:-5] not in self.census_columns_no_date and col != "cfips":
                self.census_columns_no_date.append(col[:-5])
            else: continue

        for col in self.census_columns_no_date:
            self.train_df[col] = np.nan
            
            
    def add_census_values(self, r):
        date = r.first_day_of_month
        year = date[:4]
        year_exogenous = str(int(year) - 1)
        cfips = r.cfips
        for col in self.census_columns_no_date:
            census_col = f"{col}_{year_exogenous}"
            r[col] = self.census_starter_df[census_col].loc[self.census_starter_df.cfips == cfips].iloc[0]
        return r
    
    
    def bfill_nan(self):
        columns = self.train_df.columns

        for col in columns:
            self.train_df[col] = self.train_df[col].bfill()
            
            
    def add_month_col(self):
        formatting = '%Y-%m-%d'
        self.train_df['first_day_of_month'] = pd.to_datetime(self.train_df['first_day_of_month'], format=formatting)
        self.train_df["Month"] = self.train_df['first_day_of_month'].dt.strftime('%b')
        
        
    def remove_non_feature_cols(self):
        self.train_df = self.train_df.drop(["county", "first_day_of_month", "state"], axis=1)
        
    
    def create_dummies(self):
        self.train_df = self.train_df.set_index("row_id")
        self.train_df = pd.get_dummies(self.train_df, prefix="Month")
        
               
    
    def reindex_dataset(self):
        self.train_df = self.train_df.reindex(columns=["cfips", "active", "pct_bb", "pct_college", "pct_foreign_born",
                                  "pct_it_workers", "median_hh_inc", "Month_Jan", "Month_Feb",
                                  "Month_Mar", "Month_Apr", "Month_May", "Month_Jun", "Month_Jul",
                                  "Month_Aug", "Month_Sep", "Month_Oct", "Month_Nov", "Month_Dec", "microbusiness_density"])    

    
    
    def train_model(self):
        cfips = self.train_df.cfips.unique()
        cols_mm = ["microbusiness_density", "active", "pct_bb", "pct_college", 
                   "pct_foreign_born", "pct_it_workers", "median_hh_inc"]
        for cfip in cfips:
            df_cfip = self.train_df.loc[self.train_df.cfips == cfip]
            mms_target = MinMaxScaler()
            mms_target.fit(df_cfip.microbusiness_density.to_numpy().reshape(-1, 1))
            mms = MinMaxScaler()
            df_cfip[cols_mm] = mms.fit_transform(df_cfip[cols_mm])
            features = df_cfip.drop(["microbusiness_density", "cfips"], axis=1)
            target = df_cfip.microbusiness_density
            regr = xgb.XGBRegressor(eval_metric='mape')
            clf = GridSearchCV(estimator=regr, 
                           param_grid=self.params,
                           scoring='neg_mean_squared_error', 
                           verbose=1,
                           n_jobs=-1,
                           cv=3)
            clf.fit(features, target)
            regr = xgb.XGBRegressor(**clf.best_params_)
            regr.fit(features, target)
            self.model_container[str(cfip)] = [regr, mms_target]

In [3]:
trainer = Trainer()

In [4]:
trainer.preprocess()

In [5]:
trainer.train_model()

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

...

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Fitting 3 folds for each

In [7]:
trainer.model_container["1001"]

[XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=125, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 MinMaxScaler()]

### Store the trained models per county as one pickle file

In [9]:
with open('models_per_county_GoDaddy.pickle', 'wb') as handle:
    pickle.dump(trainer.model_container, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open('models_per_county_GoDaddy.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [11]:
b["1001"]

[XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=125, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 MinMaxScaler()]